In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_vehicle = {}
list_job = []
for j in range(len(input_)):
    if input_.loc[j, 'Vehicle_Page'] != '':
        dict_vehicle[input_.loc[j, 'Item_Number']] = {}
        dict_vehicle[input_.loc[j, 'Item_Number']]['vehicle_page'] = int(input_.loc[j, 'Vehicle_Page'])
        for k in range(int(input_.loc[j, 'Vehicle_Page'])):
            list_job.append((j, k))

length = len(list_job)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(len(list_job)):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
               
        j, k = list_job[i]
        
        # = = = = = = = = = = = = = = =
                  
        crawler_status = 'error'
        for _ in range(101):
            try:
                data = '{"scopedContext":{"catalogDetails":{"itemId":"' + input_.loc[j, 'Item_Number'] + '","categoryId":"' + input_.loc[j, 'Category_Id'] + '","marketplaceId":"' + input_.loc[j, 'Market_Id'] + '"}}}'

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/json'

                resp = requests.post('https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(k*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true',
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)

                # = = = = = = = = = = = = = = =

                list_header = resp['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']

                # = = = = = = = = = = = = = = =

                for m in range(len(list_header)):
                    if 'Make' in list_header[m]['textSpans'][0]['text']:
                        col_make = m
                    elif 'Model' in list_header[m]['textSpans'][0]['text']:
                        col_model = m
                    elif 'Year' in list_header[m]['textSpans'][0]['text']:
                        col_year = m

                # = = = = = = = = = = = = = = =

                list_row = resp['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

                # = = = = = = = = = = = = = = =

                list_vehicle = []
                for row in list_row:
                    make = row['cells'][col_make]['textSpans'][0]['text'].strip()
                    model = row['cells'][col_model]['textSpans'][0]['text'].strip()
                    year = int(row['cells'][col_year]['textSpans'][0]['text'].strip())

                    list_vehicle.append({'make': make, 'model': model, 'year': year})

                dict_vehicle[input_.loc[j, 'Item_Number']][str(k)] = list_vehicle
                
                # = = = = = = = = = = = = = = =
                
                if dict_vehicle[input_.loc[j, 'Item_Number']]['vehicle_page']+1 == len(dict_vehicle[input_.loc[j, 'Item_Number']]):
                    item_list_vehicle = []
                    item_dict_vehicle = {}
                    for m in range(dict_vehicle[input_.loc[j, 'Item_Number']]['vehicle_page']):
                        item_list_vehicle += dict_vehicle[input_.loc[j, 'Item_Number']][str(m)]

                    for vehicle in item_list_vehicle:
                        if vehicle['make'] not in item_dict_vehicle:
                            item_dict_vehicle[vehicle['make']] = {vehicle['model']: [vehicle['year']]}
                        else:
                            if vehicle['model'] not in item_dict_vehicle[vehicle['make']]:
                                item_dict_vehicle[vehicle['make']][vehicle['model']] = [vehicle['year']]
                            else:
                                item_dict_vehicle[vehicle['make']][vehicle['model']].append(vehicle['year'])
                
                    # = = = = = = = = = = = = = = =
                    
                    item_list_vehicle = []
                    for make in item_dict_vehicle:
                        for model in item_dict_vehicle[make]:
                            year_max = max(item_dict_vehicle[make][model])
                            year_min = min(item_dict_vehicle[make][model])
                            if year_max == year_min:
                                item_list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                item_list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    vehicle = '\n'.join(item_list_vehicle)
                
                    # = = = = = = = = = = = = = = =

                    df_temp = pd.DataFrame([{'Item_Number': input_.loc[j, 'Item_Number'],
                                             'Vehicle': vehicle}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                    
                    print('Note：' + input_.loc[j, 'Item_Number'] + ' 已全部获取')
                    
                # = = = = = = = = = = = = = = =
               
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        print(input_.loc[j, 'Item_Number'] + ' ' + str(k) +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =         
               
list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True])
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True])
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：71841

172199068741 4  <->  [ok] - 剩余数量：71831 - 当前时间：10:04:53
172199068741 3  <->  [ok] - 剩余数量：71830 - 当前时间：10:04:53
172199068741 0  <->  [ok] - 剩余数量：71829 - 当前时间：10:04:53
172199068741 5  <->  [ok] - 剩余数量：71828 - 当前时间：10:04:53
172199068741 1  <->  [ok] - 剩余数量：71827 - 当前时间：10:04:54
172199068741 7  <->  [ok] - 剩余数量：71826 - 当前时间：10:04:54
172199068741 9  <->  [ok] - 剩余数量：71825 - 当前时间：10:04:54
172199068741 11  <->  [ok] - 剩余数量：71824 - 当前时间：10:04:54
172199068741 10  <->  [ok] - 剩余数量：71823 - 当前时间：10:04:54
172199068741 17  <->  [ok] - 剩余数量：71822 - 当前时间：10:04:55
172199068741 13  <->  [ok] - 剩余数量：71821 - 当前时间：10:04:55
172199068741 14  <->  [ok] - 剩余数量：71820 - 当前时间：10:04:55
172199068741 18  <->  [ok] - 剩余数量：71819 - 当前时间：10:04:55
172199068741 2  <->  [ok] - 剩余数量：71818 - 当前时间：10:04:56
172294135417 0  <->  [ok] - 剩余数量：71817 - 当前时间：10:04:56
172199068741 16  <->  [ok] - 剩余数量：71816 - 当前时间：10:04:56
172294135417 1  <->  [ok] - 剩余数量：71815 - 当前时间：10:04:57
172294135417 2  <->  [ok] - 剩余数量：71814 - 当前时间：1

172296542378 106  <->  [ok] - 剩余数量：71684 - 当前时间：10:05:31
172296542378 107  <->  [ok] - 剩余数量：71683 - 当前时间：10:05:31
172296542378 108  <->  [ok] - 剩余数量：71682 - 当前时间：10:05:32
172296542378 110  <->  [ok] - 剩余数量：71681 - 当前时间：10:05:32
172296542378 99  <->  [ok] - 剩余数量：71680 - 当前时间：10:05:32
172296542378 111  <->  [ok] - 剩余数量：71679 - 当前时间：10:05:33
172296542378 112  <->  [ok] - 剩余数量：71678 - 当前时间：10:05:33
172296542378 113  <->  [ok] - 剩余数量：71677 - 当前时间：10:05:35
172296542378 114  <->  [ok] - 剩余数量：71676 - 当前时间：10:05:35
172296542378 115  <->  [ok] - 剩余数量：71675 - 当前时间：10:05:35
172296542378 116  <->  [ok] - 剩余数量：71674 - 当前时间：10:05:36
172296542378 117  <->  [ok] - 剩余数量：71673 - 当前时间：10:05:36
172296542378 118  <->  [ok] - 剩余数量：71672 - 当前时间：10:05:36
172296542378 121  <->  [ok] - 剩余数量：71671 - 当前时间：10:05:37
172296542378 120  <->  [ok] - 剩余数量：71670 - 当前时间：10:05:37
172296542378 122  <->  [ok] - 剩余数量：71669 - 当前时间：10:05:37
172296542378 119  <->  [ok] - 剩余数量：71668 - 当前时间：10:05:38
172296542378 92  <->  [ok] - 剩余数

172562551580 29  <->  [ok] - 剩余数量：71538 - 当前时间：10:06:54
172562551580 21  <->  [ok] - 剩余数量：71537 - 当前时间：10:06:55
172562551580 30  <->  [ok] - 剩余数量：71536 - 当前时间：10:06:55
172562551580 31  <->  [ok] - 剩余数量：71535 - 当前时间：10:06:55
172562551580 33  <->  [ok] - 剩余数量：71534 - 当前时间：10:06:56
172562551580 35  <->  [ok] - 剩余数量：71533 - 当前时间：10:06:57
172562551580 34  <->  [ok] - 剩余数量：71532 - 当前时间：10:06:58
172562551580 36  <->  [ok] - 剩余数量：71531 - 当前时间：10:06:59
172562551580 37  <->  [ok] - 剩余数量：71530 - 当前时间：10:06:59
172562551580 38  <->  [ok] - 剩余数量：71529 - 当前时间：10:07:01
172562551580 39  <->  [ok] - 剩余数量：71528 - 当前时间：10:07:01
172562551580 40  <->  [ok] - 剩余数量：71527 - 当前时间：10:07:02
172562551580 41  <->  [ok] - 剩余数量：71526 - 当前时间：10:07:03
172562551580 42  <->  [ok] - 剩余数量：71525 - 当前时间：10:07:03
172562551580 43  <->  [ok] - 剩余数量：71524 - 当前时间：10:07:04
172562551580 44  <->  [ok] - 剩余数量：71523 - 当前时间：10:07:04
172562551580 45  <->  [ok] - 剩余数量：71522 - 当前时间：10:07:05
172562551580 46  <->  [ok] - 剩余数量：71521 - 当前时间：1

173350224844 49  <->  [ok] - 剩余数量：71392 - 当前时间：10:08:38
173350224844 51  <->  [ok] - 剩余数量：71391 - 当前时间：10:08:38
173350224844 47  <->  [ok] - 剩余数量：71390 - 当前时间：10:08:39
173350224844 52  <->  [ok] - 剩余数量：71389 - 当前时间：10:08:40
173350224844 53  <->  [ok] - 剩余数量：71388 - 当前时间：10:08:40
173350224844 54  <->  [ok] - 剩余数量：71387 - 当前时间：10:08:40
173350224844 55  <->  [ok] - 剩余数量：71386 - 当前时间：10:08:41
173350224844 56  <->  [ok] - 剩余数量：71385 - 当前时间：10:08:41
173350224844 57  <->  [ok] - 剩余数量：71384 - 当前时间：10:08:41
173350224844 58  <->  [ok] - 剩余数量：71383 - 当前时间：10:08:42
173350224844 59  <->  [ok] - 剩余数量：71382 - 当前时间：10:08:43
173350224844 60  <->  [ok] - 剩余数量：71381 - 当前时间：10:08:43
173350224844 50  <->  [ok] - 剩余数量：71380 - 当前时间：10:08:43
173350224844 61  <->  [ok] - 剩余数量：71379 - 当前时间：10:08:44
173350224844 62  <->  [ok] - 剩余数量：71378 - 当前时间：10:08:44
173350224844 63  <->  [ok] - 剩余数量：71377 - 当前时间：10:08:44
173350224844 64  <->  [ok] - 剩余数量：71376 - 当前时间：10:08:45
173350224844 67  <->  [ok] - 剩余数量：71375 - 当前时间：1

173823883928 1  <->  [ok] - 剩余数量：71247 - 当前时间：10:09:59
173823883928 3  <->  [ok] - 剩余数量：71246 - 当前时间：10:09:59
173823883928 4  <->  [ok] - 剩余数量：71245 - 当前时间：10:09:59
173579751181 69  <->  [ok] - 剩余数量：71244 - 当前时间：10:10:00
173823883928 5  <->  [ok] - 剩余数量：71243 - 当前时间：10:10:00
173823883928 7  <->  [ok] - 剩余数量：71242 - 当前时间：10:10:00
173823883928 6  <->  [ok] - 剩余数量：71241 - 当前时间：10:10:01
173823883928 9  <->  [ok] - 剩余数量：71240 - 当前时间：10:10:01
173823883928 8  <->  [ok] - 剩余数量：71239 - 当前时间：10:10:02
173823883928 11  <->  [ok] - 剩余数量：71238 - 当前时间：10:10:02
173579751181 34  <->  [ok] - 剩余数量：71237 - 当前时间：10:10:02
173823883928 12  <->  [ok] - 剩余数量：71236 - 当前时间：10:10:03
173823883928 14  <->  [ok] - 剩余数量：71235 - 当前时间：10:10:03
173823883928 15  <->  [ok] - 剩余数量：71234 - 当前时间：10:10:04
173823883928 16  <->  [ok] - 剩余数量：71233 - 当前时间：10:10:04
173823883928 17  <->  [ok] - 剩余数量：71232 - 当前时间：10:10:04
173823883928 18  <->  [ok] - 剩余数量：71231 - 当前时间：10:10:05
173823883928 19  <->  [ok] - 剩余数量：71230 - 当前时间：10:10:05


...

175837935396 117  <->  [ok] - 剩余数量：47716 - 当前时间：13:23:24
175837935396 115  <->  [ok] - 剩余数量：47715 - 当前时间：13:23:25
175837935396 118  <->  [ok] - 剩余数量：47714 - 当前时间：13:23:26
175837935396 114  <->  [ok] - 剩余数量：47713 - 当前时间：13:23:26
175837935396 119  <->  [ok] - 剩余数量：47712 - 当前时间：13:23:26
175837935396 120  <->  [ok] - 剩余数量：47711 - 当前时间：13:23:27
175837935396 121  <->  [ok] - 剩余数量：47710 - 当前时间：13:23:27
175837935396 88  <->  [ok] - 剩余数量：47709 - 当前时间：13:23:27
175837935396 122  <->  [ok] - 剩余数量：47708 - 当前时间：13:23:28
175837935396 124  <->  [ok] - 剩余数量：47707 - 当前时间：13:23:28
175837935396 125  <->  [ok] - 剩余数量：47706 - 当前时间：13:23:28
175837935396 123  <->  [ok] - 剩余数量：47705 - 当前时间：13:23:28
175837935396 126  <->  [ok] - 剩余数量：47704 - 当前时间：13:23:29
175837935396 129  <->  [ok] - 剩余数量：47703 - 当前时间：13:23:29
175837935396 128  <->  [ok] - 剩余数量：47702 - 当前时间：13:23:30
175837935396 130  <->  [ok] - 剩余数量：47701 - 当前时间：13:23:30
175837935396 131  <->  [ok] - 剩余数量：47700 - 当前时间：13:23:31
175837935396 132  <->  [ok] - 剩余

175837939678 112  <->  [ok] - 剩余数量：47571 - 当前时间：13:25:51
175837939678 113  <->  [ok] - 剩余数量：47570 - 当前时间：13:25:52
175837939678 114  <->  [ok] - 剩余数量：47569 - 当前时间：13:25:54
175837939678 115  <->  [ok] - 剩余数量：47568 - 当前时间：13:25:58
175837939678 116  <->  [ok] - 剩余数量：47567 - 当前时间：13:25:59
175837939678 117  <->  [ok] - 剩余数量：47566 - 当前时间：13:26:01
175837939678 118  <->  [ok] - 剩余数量：47565 - 当前时间：13:26:02
175837939678 119  <->  [ok] - 剩余数量：47564 - 当前时间：13:26:04
175837939678 120  <->  [ok] - 剩余数量：47563 - 当前时间：13:26:09
175837939678 121  <->  [ok] - 剩余数量：47562 - 当前时间：13:26:11
175837939678 122  <->  [ok] - 剩余数量：47561 - 当前时间：13:26:13
175837939678 110  <->  [ok] - 剩余数量：47560 - 当前时间：13:26:13
175837939678 123  <->  [ok] - 剩余数量：47559 - 当前时间：13:26:14
175837939678 124  <->  [ok] - 剩余数量：47558 - 当前时间：13:26:15
175837939678 125  <->  [ok] - 剩余数量：47557 - 当前时间：13:26:16
175837939678 126  <->  [ok] - 剩余数量：47556 - 当前时间：13:26:16
175837939678 128  <->  [ok] - 剩余数量：47555 - 当前时间：13:26:17
175837939678 127  <->  [ok] - 剩

175839135232 55  <->  [ok] - 剩余数量：47425 - 当前时间：13:28:20
Note：175839135154 已全部获取
175839135154 33  <->  [ok] - 剩余数量：47424 - 当前时间：13:28:20
175839135232 52  <->  [ok] - 剩余数量：47423 - 当前时间：13:28:21
175839135232 53  <->  [ok] - 剩余数量：47422 - 当前时间：13:28:21
175839135232 57  <->  [ok] - 剩余数量：47421 - 当前时间：13:28:21
175839135232 56  <->  [ok] - 剩余数量：47420 - 当前时间：13:28:21
175839135232 59  <->  [ok] - 剩余数量：47419 - 当前时间：13:28:21
175839135232 58  <->  [ok] - 剩余数量：47418 - 当前时间：13:28:22
175839135232 62  <->  [ok] - 剩余数量：47417 - 当前时间：13:28:22
175839135232 60  <->  [ok] - 剩余数量：47416 - 当前时间：13:28:23
175839135232 63  <->  [ok] - 剩余数量：47415 - 当前时间：13:28:23
175839135232 65  <->  [ok] - 剩余数量：47414 - 当前时间：13:28:24
175839135232 66  <->  [ok] - 剩余数量：47413 - 当前时间：13:28:25
175839135232 67  <->  [ok] - 剩余数量：47412 - 当前时间：13:28:25
175839135232 68  <->  [ok] - 剩余数量：47411 - 当前时间：13:28:26
175839135232 69  <->  [ok] - 剩余数量：47410 - 当前时间：13:28:27
175839135232 70  <->  [ok] - 剩余数量：47409 - 当前时间：13:28:27
175839135232 71  <->  [o

175839266365 14  <->  [ok] - 剩余数量：47279 - 当前时间：13:30:09
Note：175839139439 已全部获取
175839139439 8  <->  [ok] - 剩余数量：47278 - 当前时间：13:30:09
175839266365 15  <->  [ok] - 剩余数量：47277 - 当前时间：13:30:10
175839266365 16  <->  [ok] - 剩余数量：47276 - 当前时间：13:30:10
175839266365 18  <->  [ok] - 剩余数量：47275 - 当前时间：13:30:11
175839266365 20  <->  [ok] - 剩余数量：47274 - 当前时间：13:30:11
175839266365 19  <->  [ok] - 剩余数量：47273 - 当前时间：13:30:12
175839266365 21  <->  [ok] - 剩余数量：47272 - 当前时间：13:30:12
175839266365 22  <->  [ok] - 剩余数量：47271 - 当前时间：13:30:13
175839266365 23  <->  [ok] - 剩余数量：47270 - 当前时间：13:30:14
175839266365 25  <->  [ok] - 剩余数量：47269 - 当前时间：13:30:15
175839266365 17  <->  [ok] - 剩余数量：47268 - 当前时间：13:30:16
175839266365 27  <->  [ok] - 剩余数量：47267 - 当前时间：13:30:16
175839266365 28  <->  [ok] - 剩余数量：47266 - 当前时间：13:30:17
175839266365 29  <->  [ok] - 剩余数量：47265 - 当前时间：13:30:17
175839266365 30  <->  [ok] - 剩余数量：47264 - 当前时间：13:30:18
175839266365 32  <->  [ok] - 剩余数量：47263 - 当前时间：13:30:19
175839266365 33  <->  [ok

175841504179 21  <->  [ok] - 剩余数量：47134 - 当前时间：13:32:36
175841504179 23  <->  [ok] - 剩余数量：47133 - 当前时间：13:32:36
175841504179 22  <->  [ok] - 剩余数量：47132 - 当前时间：13:32:36
175841504179 25  <->  [ok] - 剩余数量：47131 - 当前时间：13:32:37
175841504179 24  <->  [ok] - 剩余数量：47130 - 当前时间：13:32:37
175841504179 27  <->  [ok] - 剩余数量：47129 - 当前时间：13:32:38
175841504179 26  <->  [ok] - 剩余数量：47128 - 当前时间：13:32:39
175841504179 29  <->  [ok] - 剩余数量：47127 - 当前时间：13:32:40
175841504179 31  <->  [ok] - 剩余数量：47126 - 当前时间：13:32:42
175841504179 32  <->  [ok] - 剩余数量：47125 - 当前时间：13:32:43
175841504179 33  <->  [ok] - 剩余数量：47124 - 当前时间：13:32:46
175841504179 34  <->  [ok] - 剩余数量：47123 - 当前时间：13:32:49
175841504179 35  <->  [ok] - 剩余数量：47122 - 当前时间：13:32:50
175841504179 36  <->  [ok] - 剩余数量：47121 - 当前时间：13:32:52
175841504179 37  <->  [ok] - 剩余数量：47120 - 当前时间：13:32:54
175841504179 38  <->  [ok] - 剩余数量：47119 - 当前时间：13:32:55
175841504179 39  <->  [ok] - 剩余数量：47118 - 当前时间：13:32:57
175841504179 40  <->  [ok] - 剩余数量：47117 - 当前时间：1